# Setup

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import datetime
from src import (get_data as gd, 
                 prepare as prep, 
                 aggregate_data as aggregate,
                 push_data,
                 sedi_rate,
                 age_sr_plot)
###
orig_dir = os.getcwd()

# Main Data

### Retrieve data 

In [ ]:
dates = gd.AgeFromFileOneCore()

In [ ]:
dates.get_dates()

### Add calibration curve to data

In [ ]:
dates.select_calibration_curve(default_curve = 'IntCal20', user_selection = False)

In [ ]:
dates.add_calibration_curve()

### Put data in variables

In [ ]:
AllAges = dates.all_ages
CoreIDs = dates.all_coreid_list
CoreLengths = dates.all_core_lengths
SR_mode = 'naive'

In [ ]:
aggRC = lambda: None
aggRC.dttp = "No"

# Reservoir correction

#### Prep

In [ ]:
os.chdir(orig_dir)
RC = prep.PrepForReservoirCorrection(all_ages = AllAges)
RC.prep_it()
RC_Frame = RC.RC_Frame
RC_CoreIDs = RC.RC_CoreIDs
RC_CoreLengths = CoreLengths[CoreLengths.coreid.isin(RC_CoreIDs)]

#### R run

In [ ]:
%get RC_Frame
%get RC_CoreIDs 
%get RC_CoreLengths

In [ ]:
K_fine <- 100
source('src/Run_DefineReservoir.R', local = TRUE)

In [ ]:
%put reservoir_core_results

#### Aggregate results

In [ ]:
aggRC = aggregate.AggDataReservoir(results = reservoir_core_results,
                                   surface_dates = RC.desired_surface_dates,
                                   verbose = 0)
aggRC.results_agg()

In [ ]:
AllAges = aggRC.add_reservoir(all_ages = AllAges)

# Calibration
#### Prep

In [ ]:
calib = prep.PrepForCalibration(all_ages = AllAges)
calib.prep_it()
calib_Frame = calib.calib_Frame

#### R run

In [ ]:
%get calib_Frame --from SoS

In [ ]:
source('src/Run_calibration.R', local = TRUE)

In [ ]:
%put calib_dates

# Age Modeling
### UNDATABLE
#### Prep

In [ ]:
Undatable = prep.PrepForUndatable(all_ages = AllAges, 
                                  all_coreid_list = CoreIDs)
Undatable.prep_it()
CoreID_array = Undatable.CoreID_array

#### Octave run

In [ ]:
%get CoreID_array

In [ ]:
xfactor = 0.1;
bootpc = 30;
run_Undatable(CoreID_array, xfactor, bootpc);

#### Aggregate results

In [ ]:
aggU = aggregate.AggDataUndatable(Undatable, orig_dir, dttp = aggRC.dttp)
aggU.results_agg()

#### Calculate sedimentation rate

In [ ]:
SRUn = sedi_rate.CalculateSediRate(aggU, model = 'Undatable', coreid = CoreIDs, mode = SR_mode)
SRUn.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggU,
                        engine = dates.engine,
                        model = 'Undatable')
push.push_to_db()

In [ ]:
os.chdir(orig_dir)
push.delete_files(location_UndatableFolder = Undatable.location_UndatableFolder,
                  coreids = Undatable.coreid_df)

### Bchron

#### Prep

In [ ]:
os.chdir(orig_dir)
Bchron = prep.PrepForBchron(all_ages = AllAges)
Bchron.prep_it()
Bchron_Frame = Bchron.Bchron_Frame

#### R run

In [ ]:
%get Bchron_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [ ]:
source('src/Run_Bchron.R')

In [ ]:
%put Bchron_core_results

#### Aggregate results

In [ ]:
aggBc = aggregate.AggDataBchron(Bchron_core_results, dttp = aggRC.dttp)
aggBc.results_agg()

#### Calculating sedimentation rate

In [ ]:
SRBc = sedi_rate.CalculateSediRate(aggBc, model = 'Bchron', coreid = CoreIDs, mode = SR_mode)
SRBc.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggBc,
                        engine = dates.engine,
                        model = 'Bchron')
push.push_to_db()

### hamstr

#### Prep

In [ ]:
hamstr = prep.PrepForHamstr(all_ages = AllAges)
hamstr.prep_it()
hamstr_Frame = hamstr.hamstr_Frame

#### R Run

In [ ]:
%get hamstr_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [ ]:
K_fine <- 100
source('src/Run_hamstr.R', local = TRUE)

In [ ]:
%put hamstr_core_results

#### Aggregate

In [ ]:
aggh = aggregate.AggDataHamstr(hamstr_core_results, dttp = aggRC.dttp)
aggh.results_agg()

#### Calculating sedimentation rate

In [ ]:
SRh = sedi_rate.CalculateSediRate(aggh, model = 'hamstr', coreid = CoreIDs, mode = SR_mode)
SRh.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggh,
                        engine = dates.engine,
                        model = 'hamstr')
push.push_to_db()

### Bacon

#### Prep

In [ ]:
Bacon = prep.PrepForBacon(all_ages = AllAges)
Bacon.prep_it()
Bacon_Frame = Bacon.Bacon_Frame

#### R Run

In [ ]:
%get Bacon_Frame --from SoS 
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [ ]:
acc.shape <- 1.5
acc.mean <- 20
rbacon.change.acc.mean <- TRUE
rbacon.change.thick <- TRUE
mem.strength <- 10
mem.mean <- 0.5
ssize <- 10000
source('src/Run_Bacon.R', local = TRUE)

In [ ]:
%put Bacon_core_results

#### Aggregate

In [ ]:
aggBa = aggregate.AggDataBacon(Bacon_core_results, dttp = aggRC.dttp)
aggBa.results_agg()

starting worker for localhost:11609 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 
Type: EXEC 


Loading required package: foreach
Loading required package: rngtools
Running Bacon for core EN18218 (attempt 1, ssize = 10000)
 Using a mix of cal BP and calibrated C-14 dates

Adapting acc.mean to 50 yr/cm
 Will run 93,800,000 iterations and store 10,000


The run's files will be put in this folder: /tmp/RtmpIcWpLG/EN18218
Reading /tmp/RtmpIcWpLG/EN18218/EN18218_132.bacon
Constant calibration curve.
IntCal20: Reading from file: /opt/conda/lib/R/library/rintcal/extdata/3Col_intcal20.14C
Marine20: Reading from file: /opt/conda/lib/R/library/rintcal/extdata/3Col_marine20.14C
SHCal20: Reading from file: /opt/conda/lib/R/library/rintcal/extdata/3Col_shcal20.14C
Added det: EN18218:  -68.0+-5.0    d=0 ResCorr=   0.0+-0.0    a=3 b=4 cc=Constant c. curve
Added det: EN18218:  784.0+-30.0   d=0.25 ResCorr= 853.0+-31.0   a=3 b=4 cc=Constant c. curve
Added det: EN18218: 2787.0+-33.0   d=18.75 ResCorr= 853.0+-31.0   a=3 b=4 cc=IntCal20
Added det: EN18218: 3628.0+-32.0   d=35.25 ResCorr= 853.0+-31.0   a=3 b=4 cc=IntCal20
Added det: EN18218: 3832.0+-32.0   d=59.75 ResCorr= 853.0+-31.0   a=3 b=4 cc=IntCal20
Added det: EN18218: 4985.0+-33.0   d=81.25 ResCorr= 853.0+-31.0   a=3 b=4 cc=IntCal20
Added det: EN18218: 5417.0+-33.0   d=114.75 ResCorr= 853.0+-31.

Warning, this will take quite some time to calculate. I suggest increasing d.by to, e.g., 10

Calculating age ranges...


  |======================================================================| 100%


Preparing ghost graph... 


  |=======                                                               |  11%

#### Calculating sedimentation rate

In [ ]:
SRBa = sedi_rate.CalculateSediRate(aggBa, model = 'Bacon', coreid = CoreIDs, mode = SR_mode)
SRBa.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggBa,
                        engine = dates.engine,
                        model = 'Bacon')
push.push_to_db()

### Clam

#### Prep

In [ ]:
clam = prep.PrepForClam(all_ages = AllAges)
clam.prep_it()
clam_Frame = clam.clam_Frame

#### R Run

In [ ]:
%get clam_Frame --from SoS
%get CoreIDs --from SoS
%get CoreLengths --from SoS

In [ ]:
types_curve <- 1:5
smoothness_curve <- 0.1*(1:10)
poly_degree_curve <- 1:4
best_fit <- TRUE

In [ ]:
source('src/Run_clam.R', local = TRUE)

In [ ]:
clam_core_results

In [ ]:
%put clam_core_results

#### Aggregate

In [ ]:
aggcl = aggregate.AggDataClam(clam_core_results, dttp = aggRC.dttp)
aggcl.results_agg()

#### Calculating sedimentation rate

In [ ]:
SRcl = sedi_rate.CalculateSediRate(aggcl, model = 'clam', coreid = CoreIDs, mode = SR_mode)
SRcl.calculating_SR()

#### Push results to database

In [ ]:
push = push_data.PushIt(agg = aggcl,
                        engine = dates.engine,
                        model = 'clam')
push.push_to_db()

# Plotting

In [ ]:
plot_data = {'Undatable' : [aggU.age_model_result_Undatable, SRUn.SR_model_result_Undatable],
             'Bchron': [aggBc.age_model_result_Bchron, SRBc.SR_model_result_Bchron],
             'hamstr': [aggh.age_model_result_hamstr, SRh.SR_model_result_hamstr],
             'Bacon': [aggBa.age_model_result_Bacon, SRBa.SR_model_result_Bacon],
             'clam': [aggcl.age_model_result_clam, SRcl.SR_model_result_clam],
             'calib_dates': calib_dates}

In [ ]:
ASRplot = age_sr_plot.PlotAgeSR(plot_data, coreid = CoreIDs, dttp = aggRC.dttp)

In [ ]:
ASRplot.plot_graph(orig_dir = orig_dir, sigma_range = 'both',
                   bin_size = 1000, xlim_max = 21000, number_col = 8, reduce_plot_axis = True,
                   only_combined = True, save = True, for_color_blind = False, as_jpg = False)

# Optimization

In [ ]:
proxy = gd.ProxyFromFile()

In [ ]:
proxy_ts = proxy.get_proxy()
###
dict_model_name = ASRplot.dict_model_name
dict_SR_median_age = ASRplot.dict_SR_median_age

In [ ]:
%get dict_SR_median_age --from SoS
%get dict_model_name --from SoS
%get proxy_ts --from SoS

In [ ]:
curr_nreps     <- 1000
curr_minseg    <- 50
curr_cptmethod <- "mean+variance"

In [ ]:
source('src/Run_Optimization.R', local = TRUE)

In [ ]:
%put result_list 
%put fitting_values

In [ ]:
optimizedASRplot = age_sr_plot.PlotAgeSR(plot_data, coreid = CoreIDs, dttp = aggRC.dttp)

In [ ]:
optimizedASRplot.plot_optimized_graph(optimization_values = result_list, fitting_values = fitting_values, 
                                      proxy = proxy.name, proxy_data = proxy_ts, orig_dir = orig_dir, 
                                      sigma_range = 'both', inclusion_threshold = 0.03, show_fitting_models = True,
                                      bin_size = 1000, xlim_max = 21000, number_col = 8, reduce_plot_axis = True,
                                      only_combined = True, save = True, for_color_blind = False, as_jpg = False)